In [1]:
## import some various libraries
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import re
import json
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

import unidecode

import urllib.request

import warnings
warnings.filterwarnings("ignore")
import argparse
import logging
import time
from sqlalchemy import create_engine

import holidays

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [2]:
### for this task i used the provided DB as the traffic limitations only let me 15 records at a time with the app

username = 'postgres'
password = 'Mutt2023'
host = '0.0.0.0'  
port = '5432'  
database_name = 'postgres'


engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database_name}')

In [3]:
# Connect to the database
connection = engine.connect()

In [4]:
sql_query = "select *  from coin_data"

dfsql = pd.read_sql_query(sql_query, engine)
dfsql['date'] = pd.to_datetime(dfsql['date'])

    


In [5]:
def volumetransacted(row):
    return row['json']['market_data']['total_volume']['usd']
    

dfsql['volume_transacted'] = dfsql.apply (lambda row: volumetransacted(row), axis=1)

In [6]:
df = dfsql

In [7]:
df = df[['coin', 'date', 'price', 'volume_transacted']]

In [8]:
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by=['coin', 'date'], ascending=False)

df['risk_level'] = 'low risk'

In [9]:
### creating 3 separate dfs. one per coin

unique_coins = df['coin'].unique()

coin_dfs = {}

for coin in unique_coins:
    coin_dfs[coin] = df[df['coin'] == coin].reset_index(drop=True)

In [10]:
### Creating a func that will iterate per row checking to create all different featuresk


In [11]:
for j in unique_coins:
    
    ### adding the feature "risk_level"
    
    def check_risk(row):
        try:
            index = row.name
#             print(row['price'])
#             print(coin_dfs[j].iloc[index + 1]['price'])
            today_price = row['price']
            yesterday_price1 = coin_dfs[j].iloc[index + 1]['price']
            yesterday_price2 = coin_dfs[j].iloc[index + 2]['price']

            percentage_decrease1 = ((yesterday_price1 - today_price) / yesterday_price1) * 100
            percentage_decrease2 = ((yesterday_price2 - yesterday_price1) / yesterday_price2) * 100
            
            if percentage_decrease1 > 50:
                    if percentage_decrease2 > 50:
                        return 'high risk'
            elif percentage_decrease1 > 20:
                    if percentage_decrease2 > 20:
                        return 'medium risk'
            return 'low risk'
        except: 
            return np.nan
    
    
    coin_dfs[j]['risk_level'] = coin_dfs[j].apply (lambda row: check_risk(row), axis=1)
    
    ### adding a feture per last 7 days to adjust tendency
    
    for i in range(7):
        feature_name = f'T-{i+1}'

        def dayanalysis(row):
            try:
                index = row.name
                price1 = coin_dfs[j].iloc[index + i]['price']
                price2 = coin_dfs[j].iloc[index + 1 + i]['price']
                if price2 > price1:
                    return 1
                else:
                    return 0
            except:
                return np.nan

        coin_dfs[j][feature_name] = coin_dfs[j].apply (lambda row: dayanalysis(row), axis=1)

    ### setting the variance for the last 7 days 
    coin_dfs[j]['price_variance'] = coin_dfs[j]['price'].rolling(window=8, min_periods=1).var()
    
    ### getting the values for the last 7 days
    
    for i in range(1,8):
        feature_name = f'T-{i}_value'

        def pastdaysvalues(row):
            try:
                index = row.name
                nextprice = coin_dfs[j].iloc[index + i]['price']
                return nextprice
            except:
                return np.nan

        coin_dfs[j][feature_name] = coin_dfs[j].apply (lambda row: pastdaysvalues(row), axis=1)
        
        
    ### getting the target varuiable
    
    feature_name = f'T+1_value'

    def pastdaysvalues(row):
        try:
            index = row.name
            futureprice = coin_dfs[j].iloc[index - 1]['price']
            return futureprice
        except:
            return np.nan

    coin_dfs[j][feature_name] = coin_dfs[j].apply (lambda row: pastdaysvalues(row), axis=1)
    coin_dfs[j][feature_name][0] = np.nan
    
    ### Creating new features based on the date
    
    coin_dfs[j]['month'] = coin_dfs[j]['date'].dt.month
    coin_dfs[j]['week_number'] = coin_dfs[j]['date'].dt.week
    coin_dfs[j]['day_of_week'] = coin_dfs[j]['date'].dt.dayofweek
    coin_dfs[j]['is_weekend'] = coin_dfs[j]['day_of_week'].isin([5, 6])
    
    ### Creating new features based on holidays library
    
    us_holidays = holidays.US()
    china_holidays = holidays.China()

    coin_dfs[j]['is_us_holiday'] = coin_dfs[j]['date'].apply(lambda x: x in us_holidays)
    coin_dfs[j]['is_china_holiday'] = coin_dfs[j]['date'].apply(lambda x: x in china_holidays)
    
    ### create 4 categories for each quarter of the volume transacted
    bins = [0,  coin_dfs[j]['volume_transacted'].quantile(0.25), 
                coin_dfs[j]['volume_transacted'].quantile(0.5), 
                coin_dfs[j]['volume_transacted'].quantile(0.75), 
                coin_dfs[j]['volume_transacted'].max()]
    labels = ['Q1', 'Q2', 'Q3', 'Q4']

    coin_dfs[j]['volume_transacted_Q'] = pd.cut(coin_dfs[j]['volume_transacted'], bins=bins, labels=labels, include_lowest=True)
    coin_dfs[j] = coin_dfs[j].drop(columns=['volume_transacted'])
    
    ### cleaning the DF from all nans
    
    coin_dfs[j] = coin_dfs[j].dropna()    
    
    ### Creating PCA features from the previous days values
    
    columns_to_pca = ['T-1_value', 'T-2_value','T-3_value','T-4_value','T-5_value','T-6_value','T-7_value']
    X = coin_dfs[j][columns_to_pca]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    n_components = 2
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X_scaled)
    df_pca = pd.DataFrame(data=X_pca, columns=[f'PC{i+1}' for i in range(n_components)])
    
    ### not dropping the original values as they are required in the next exercise
    
    ##coin_dfs[j] = coin_dfs[j].drop(columns=columns_to_pca)

    coin_dfs[j]['PC1_Tvalues'] = df_pca['PC1']
    coin_dfs[j]['PC2_Tvalues'] = df_pca['PC2']
    
    ### cleaning the DF from all nans
    
    coin_dfs[j] = coin_dfs[j].dropna()
    
        

In [12]:
coin_dfs['bitcoin']

,coin,date,price,risk_level,T-1,T-2,T-3,T-4,T-5,T-6,...,T+1_value,month,week_number,day_of_week,is_weekend,is_us_holiday,is_china_holiday,volume_transacted_Q,PC1_Tvalues,PC2_Tvalues
1,bitcoin,2021-09-29,41010.375198,low risk,1.0,1.0,0.0,0.0,1.0,0.0,...,41587.667862,9,39,2,False,False,False,Q1,-0.513515,-0.083770
2,bitcoin,2021-09-28,42247.355729,low risk,1.0,0.0,0.0,1.0,0.0,0.0,...,41010.375198,9,39,1,False,False,False,Q1,-0.352958,0.181224
3,bitcoin,2021-09-27,43336.861233,low risk,0.0,0.0,1.0,0.0,0.0,1.0,...,42247.355729,9,39,0,False,False,False,Q1,-0.132471,0.432094
4,bitcoin,2021-09-26,42856.855164,low risk,0.0,1.0,0.0,0.0,1.0,1.0,...,43336.861233,9,38,6,True,False,False,Q1,0.054012,0.535892
5,bitcoin,2021-09-25,42752.168155,low risk,1.0,0.0,0.0,1.0,1.0,1.0,...,42856.855164,9,38,5,True,False,False,Q2,0.177506,0.715604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,bitcoin,2021-01-13,33938.335878,low risk,1.0,1.0,1.0,1.0,0.0,0.0,...,37456.007725,1,2,2,False,False,False,Q4,-2.130150,-0.727211
261,bitcoin,2021-01-12,35669.906687,low risk,1.0,1.0,1.0,0.0,0.0,0.0,...,33938.335878,1,2,1,False,False,False,Q4,-2.358092,-0.933312
262,bitcoin,2021-01-11,38397.895985,low risk,1.0,1.0,0.0,0.0,0.0,0.0,...,35669.906687,1,2,0,False,False,False,Q4,-2.699244,-0.886663
263,bitcoin,2021-01-10,40296.529004,low risk,1.0,0.0,0.0,0.0,0.0,1.0,...,38397.895985,1,1,6,True,False,False,Q3,-3.173951,-0.783554


In [13]:
coin_dfs['cardano']

,coin,date,price,risk_level,T-1,T-2,T-3,T-4,T-5,T-6,...,T+1_value,month,week_number,day_of_week,is_weekend,is_us_holiday,is_china_holiday,volume_transacted_Q,PC1_Tvalues,PC2_Tvalues
1,cardano,2021-09-29,2.037869,low risk,1.0,1.0,1.0,0.0,1.0,0.0,...,2.058527,9,39,2,False,False,False,Q1,3.354019,0.295175
2,cardano,2021-09-28,2.138879,low risk,1.0,1.0,0.0,1.0,0.0,0.0,...,2.037869,9,39,1,False,False,False,Q1,3.397389,0.233887
3,cardano,2021-09-27,2.212884,low risk,1.0,0.0,1.0,0.0,0.0,1.0,...,2.138879,9,39,0,False,False,False,Q2,3.431118,-0.036469
4,cardano,2021-09-26,2.311934,low risk,0.0,1.0,0.0,0.0,1.0,1.0,...,2.212884,9,38,6,True,False,False,Q3,3.476086,-0.262378
5,cardano,2021-09-25,2.271252,low risk,1.0,0.0,0.0,1.0,1.0,1.0,...,2.311934,9,38,5,True,False,False,Q3,3.534414,-0.547790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,cardano,2021-01-13,0.287840,low risk,0.0,1.0,1.0,0.0,0.0,1.0,...,0.311350,1,2,2,False,False,False,Q2,-4.531962,0.017156
261,cardano,2021-01-12,0.276512,low risk,1.0,1.0,0.0,0.0,1.0,0.0,...,0.287840,1,2,1,False,False,False,Q2,-4.591444,0.084116
262,cardano,2021-01-11,0.306214,low risk,1.0,0.0,0.0,1.0,0.0,0.0,...,0.276512,1,2,0,False,False,False,Q2,-4.682753,0.116140
263,cardano,2021-01-10,0.333070,low risk,0.0,0.0,1.0,0.0,0.0,0.0,...,0.306214,1,1,6,True,False,False,Q2,-4.761052,0.129047


In [14]:
coin_dfs['ethereum']

,coin,date,price,risk_level,T-1,T-2,T-3,T-4,T-5,T-6,...,T+1_value,month,week_number,day_of_week,is_weekend,is_us_holiday,is_china_holiday,volume_transacted_Q,PC1_Tvalues,PC2_Tvalues
1,ethereum,2021-09-29,2798.984417,low risk,1.0,1.0,0.0,0.0,1.0,0.0,...,2855.611731,9,39,2,False,False,False,Q1,2.292276,-0.097596
2,ethereum,2021-09-28,2939.742283,low risk,1.0,0.0,0.0,1.0,0.0,0.0,...,2798.984417,9,39,1,False,False,False,Q1,2.423009,0.202076
3,ethereum,2021-09-27,3063.316345,low risk,0.0,0.0,1.0,0.0,0.0,1.0,...,2939.742283,9,39,0,False,False,False,Q2,2.659942,0.437864
4,ethereum,2021-09-26,2946.970846,low risk,0.0,1.0,0.0,0.0,1.0,1.0,...,3063.316345,9,38,6,True,False,False,Q1,2.900907,0.565046
5,ethereum,2021-09-25,2930.742706,low risk,1.0,0.0,0.0,1.0,1.0,1.0,...,2946.970846,9,38,5,True,False,False,Q2,3.109189,0.871516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,ethereum,2021-01-13,1045.406815,low risk,1.0,1.0,1.0,0.0,1.0,0.0,...,1132.015591,1,2,2,False,False,False,Q3,-4.017827,-0.170280
261,ethereum,2021-01-12,1092.914338,low risk,1.0,1.0,0.0,1.0,0.0,0.0,...,1045.406815,1,2,1,False,False,False,Q4,-4.168237,-0.263833
262,ethereum,2021-01-11,1267.731003,low risk,1.0,0.0,1.0,0.0,0.0,0.0,...,1092.914338,1,2,0,False,False,False,Q4,-4.419990,-0.403601
263,ethereum,2021-01-10,1282.979576,low risk,0.0,1.0,0.0,0.0,0.0,0.0,...,1267.731003,1,1,6,True,False,False,Q3,-4.667823,-0.518172


In [15]:
newdf = coin_dfs['bitcoin'].append(coin_dfs['cardano']).append(coin_dfs['ethereum']).reset_index(drop=True)
newdf

,coin,date,price,risk_level,T-1,T-2,T-3,T-4,T-5,T-6,...,T+1_value,month,week_number,day_of_week,is_weekend,is_us_holiday,is_china_holiday,volume_transacted_Q,PC1_Tvalues,PC2_Tvalues
0,bitcoin,2021-09-29,41010.375198,low risk,1.0,1.0,0.0,0.0,1.0,0.0,...,41587.667862,9,39,2,False,False,False,Q1,-0.513515,-0.083770
1,bitcoin,2021-09-28,42247.355729,low risk,1.0,0.0,0.0,1.0,0.0,0.0,...,41010.375198,9,39,1,False,False,False,Q1,-0.352958,0.181224
2,bitcoin,2021-09-27,43336.861233,low risk,0.0,0.0,1.0,0.0,0.0,1.0,...,42247.355729,9,39,0,False,False,False,Q1,-0.132471,0.432094
3,bitcoin,2021-09-26,42856.855164,low risk,0.0,1.0,0.0,0.0,1.0,1.0,...,43336.861233,9,38,6,True,False,False,Q1,0.054012,0.535892
4,bitcoin,2021-09-25,42752.168155,low risk,1.0,0.0,0.0,1.0,1.0,1.0,...,42856.855164,9,38,5,True,False,False,Q2,0.177506,0.715604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787,ethereum,2021-01-13,1045.406815,low risk,1.0,1.0,1.0,0.0,1.0,0.0,...,1132.015591,1,2,2,False,False,False,Q3,-4.017827,-0.170280
788,ethereum,2021-01-12,1092.914338,low risk,1.0,1.0,0.0,1.0,0.0,0.0,...,1045.406815,1,2,1,False,False,False,Q4,-4.168237,-0.263833
789,ethereum,2021-01-11,1267.731003,low risk,1.0,0.0,1.0,0.0,0.0,0.0,...,1092.914338,1,2,0,False,False,False,Q4,-4.419990,-0.403601
790,ethereum,2021-01-10,1282.979576,low risk,0.0,1.0,0.0,0.0,0.0,0.0,...,1267.731003,1,1,6,True,False,False,Q3,-4.667823,-0.518172


In [16]:
### to train the models to predict there are 2 main options... the best one would be a time series using the 
### dates for stacionality. In the case below im going to be using a Random Forest with months and 
### week_number to adress stacionality

### there other option to take into consideration is the dataset to train with...
### we can train the coins sepparatedly or all together...

In [17]:
def train_RF(X_train, y_train):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    
    # use of a grid search to get the best trained model
    
# ## Opcion rapida de test
#     param_grid = {
#         'n_estimators': [100],#[100, 200],
#         'max_depth': [None],#[None, 10, 20],
#         'min_samples_split': [2],#[2, 5, 10],
#         'min_samples_leaf': [2]#[1, 2, 4]
#     }


### Opcion lenta y completa  
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [None, 10, 5],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }


    rf = RandomForestRegressor(verbose=False, n_jobs = -1)

    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, scoring='f1', cv=5)
    grid_search.fit(X_train_scaled, y_train)
    
    best_RF = grid_search.best_estimator_

    return best_RF

In [18]:
### all models are saved in the best_model dict


best_model = {}


for j in unique_coins:
    
    ### Getting all values to train
    
    y_train = coin_dfs[j]['T+1_value']
    
    numerical_columns = ['price', 'price_variance', 
                         'T-1_value', 'T-2_value',
                         'T-3_value','T-4_value',
                         'T-5_value','T-6_value',
                         'T-7_value', 'PC1_Tvalues',
                         'PC2_Tvalues']

    categorical_columns = ['risk_level', 'T-1', 'T-2', 'T-3', 'T-4',
                           'T-5', 'T-6', 'T-7',  'month',
                           'week_number', 'day_of_week', 'is_weekend', 'is_us_holiday',
                           'is_china_holiday']

    numerical_features = coin_dfs[j][numerical_columns].copy()
    categorical_features = coin_dfs[j][categorical_columns].copy()

    ### converting values to string in order to use dummies

    categorical_features['month'] = categorical_features['month'].astype(str)
    categorical_features['week_number'] = categorical_features['week_number'].astype(str)
    categorical_features['day_of_week'] = categorical_features['day_of_week'].astype(str)

    categorical_encoded = pd.get_dummies(categorical_features, drop_first=True)

    X_train = pd.concat([numerical_features, categorical_encoded], axis=1)
    ### Training with func on top
    
    best_model[j] = train_RF(X_train, y_train)

In [19]:
### training the model with all the coins together

### Getting all values to train

y_train = newdf['T+1_value']

numerical_columns = ['price', 'price_variance', 
                     'T-1_value', 'T-2_value',
                     'T-3_value','T-4_value',
                     'T-5_value','T-6_value',
                     'T-7_value', 'PC1_Tvalues',
                     'PC2_Tvalues']

categorical_columns = ['coin', 'risk_level', 'T-1', 'T-2', 'T-3', 'T-4',
                       'T-5', 'T-6', 'T-7',  'month',
                       'week_number', 'day_of_week', 'is_weekend', 'is_us_holiday',
                       'is_china_holiday']

numerical_features = newdf[numerical_columns].copy()
categorical_features = newdf[categorical_columns].copy()

### converting values to string in order to use dummies

categorical_features['month'] = categorical_features['month'].astype(str)
categorical_features['week_number'] = categorical_features['week_number'].astype(str)
categorical_features['day_of_week'] = categorical_features['day_of_week'].astype(str)

categorical_encoded = pd.get_dummies(categorical_features, drop_first=True)

X_train = pd.concat([numerical_features, categorical_encoded], axis=1)
### Training with func on top

best_model['alltogether'] = train_RF(X_train, y_train)

In [21]:
best_model['alltogether'].predict(X_train)

array([60674.83144111, 61022.70432203, 61121.5393716 , 60512.27295158,
       60158.26117355, 61039.0422787 , 61036.37613666, 60643.54488078,
       60661.73320448, 61018.73528608, 60294.97602068, 60190.03913217,
       61022.70432203, 60840.21612302, 61015.96206785, 60995.38679765,
       60777.33733123, 60263.16951947, 60238.29997982, 60856.37733862,
       60759.95780281, 60776.63324984, 60869.32629351, 61156.44408126,
       60627.84188449, 60584.33794989, 60736.29061567, 61039.0422787 ,
       60770.44394425, 60942.58913674, 60683.51258264, 60246.71307431,
       60589.27374112, 60749.31572546, 60740.26630032, 60731.61749833,
       61152.47504531, 61048.31208234, 60490.39471281, 60539.32219838,
       60732.32157972, 60998.89479527, 60679.54354669, 60942.58913674,
       60683.51258264, 60533.83086206, 60266.98288203, 60723.56731134,
       60818.80282149, 60772.66421389, 61080.410047  , 61040.40254916,
       60584.33794989, 60539.32219838, 60732.32157972, 60998.89479527,
      

In [20]:
# Close the connection when done
connection.close()